In [112]:
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from gensim import utils
import os
import nltk
import scipy.sparse as ssp
from sklearn.decomposition import TruncatedSVD
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from nltk import pos_tag

In [90]:
train=pd.read_csv("checkpoints_databases/nw_working_train_0.csv",encoding="utf8")
test=pd.read_csv("checkpoints_databases/nw_working_test_0.csv",encoding="utf8")

In [91]:
train_cl=train.drop(["Variation","Text","Class","Gene"],axis=1)
test_cl=test.drop(["Text","Class","Variation","Gene"],axis=1)
train_cl=train_cl.reset_index()
train_cl=train_cl.drop("ID",axis=1)
train_cl=train_cl.rename(columns={"index":"ID"})

In [92]:
train_cl.to_csv("nw_meta_features/meta_train_l1l2.csv")
test_cl.to_csv("nw_meta_features/meta_test_l1l2.csv")

In [138]:
data_all=pd.concat((train,test)).reset_index(drop=True)

In [132]:
stop = set(stopwords.words('english'))
exclude = set('!"#$%&\'()*+:;<=>?@[\\]^_`{|}~0123456789') 
lemma = WordNetLemmatizer()
def clean(doc,lemmatiz=False):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free_0 = [re.sub(",|\.|/"," ",ch) for ch in stop_free]
    if lemmatiz:
        punc_free_lem="".join(ch for ch in punc_free_0 if ch not in exclude)
        for word,tag in pos_tag(word_tokenize(punc_free_lem)):
            lem=[]
            wntag=tag[0].lower()
            wntag=wntag if wntag in ["a","r","n","v"] else None
            if not wntag:
                lem.append(word)
            else:
                lem.append(lemma.lemmatize(word,wntag))
            normalized=" ".join(word for word in lem)
        return normalized
    else:
        punc_free = "".join(ch for ch in punc_free_0 if ch not in exclude)
        return punc_free

In [133]:
#No lemmatization for the moment, be careful not to lemmatize then w2vec
data_all.Text = [clean(doc,True) for doc in data_all.Text]  

In [143]:
stop_free = " ".join([i for i in data_all.Text[0].lower().split() if i not in stop])
punc_free_0 = [re.sub(",|\.|/"," ",ch) for ch in stop_free]

punc_free_lem="".join(ch for ch in punc_free_0 if ch not in exclude)
lem=[]
for word,tag in pos_tag(word_tokenize(punc_free_lem)):
    
    wntag=tag[0].lower()
    wntag=wntag if wntag in ["a","r","n","v"] else None
    if not wntag:
        lem.append(word)
    else:
        lem.append(lemma.lemmatize(word,wntag))
        normalized=" ".join(word for word in lem)


In [145]:
normalized

'cyclin-dependent kinase cdks regulate variety fundamental cellular process cdk stand one last orphan cdks activate cyclin identify kinase activity reveal previous work show cdk silencing increase ets v-ets erythroblastosis virus e oncogene homolog -driven activation mapk pathway confers tamoxifen resistance breast cancer cell precise mechanisms cdk modulates ets activity generally function cdk remain elusive demonstrate cdk cyclin-dependent kinase identify cyclin activate cyclin cyclin m orphan cyclin product fama whose mutation cause star syndrome human developmental anomaly whose feature include toe syndactyly telecanthus anogenital renal malformation show star syndrome-associated cyclin mutant unable interact cdk cyclin silence phenocopies cdk silence increase c-raf conferring tamoxifen resistance breast cancer cell cdk cyclin phosphorylates ets vitro cell positively control ets degradation proteasome ets protein level increase cell derive star patient increase attributable decreas

In [ ]:
train = data_all.iloc[:len(train)]
test = data_all.iloc[len(train):]

# Here we will add features from word2vec retrained then get the mean for the sentences

In [84]:
class MySentences(object):
    """MySentences is a generator to produce a list of tokenized sentences 
    
    Takes a list of numpy arrays containing documents.
    
    Args:
        arrays: List of arrays, where each element in the array contains a document.
    """
    def __init__(self, *arrays):
        self.arrays = arrays
 
    def __iter__(self):
        for array in self.arrays:
            for document in array:
                for sent in nltk.sent_tokenize(document):
                    yield nltk.word_tokenize(sent)

def get_word2vec(sentences, location,size):
    """Returns trained word2vec
    
    Args:
        sentences: iterator for sentences
        
        location (str): Path to save/load word2vec
    """
    if os.path.exists(location):
        print('Found {}'.format(location))
        model = gensim.models.Word2Vec.load(location)
        return model
    
    print('{} not found. training model'.format(location))
    model = gensim.models.Word2Vec(sentences, size=size, window=5, min_count=5, workers=4)
    print('Model done training. Saving to disk')
    model.save(location)
    return model

In [4]:
#It's important to remove duplicated spaces for word2vec learning !
train["Text"]=[" ".join(doc.split()) for doc in train["Text"].values]
test["Text"]=[" ".join(doc.split()) for doc in test["Text"].values]

In [85]:
number_w2v=[100,200,300]
w2v={}
for size in number_w2v:
    w2v["w2v_"+str(size)] = get_word2vec(
        MySentences(
            train["Text"].values),"w2v_features"+str(size),size
    )

w2v_features100 not found. training model
Model done training. Saving to disk
w2v_features200 not found. training model
Model done training. Saving to disk
w2v_features300 not found. training model
Model done training. Saving to disk


In [86]:
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [87]:
mean_embedding_vectorizer={}
mean_embedded_train={}
mean_embedded_test={}
for name in w2v:
    mean_embedding_vectorizer[name] = MeanEmbeddingVectorizer(w2v[name])
    mean_embedded_train[name] = mean_embedding_vectorizer[name].fit_transform(train['Text'])
    mean_embedded_test[name] = mean_embedding_vectorizer[name].fit_transform(test['Text'])
df_embed_tr={}
df_embed_te={}
for name in w2v:
    df_embed_tr[name]=pd.DataFrame(mean_embedded_train[name])
    df_embed_te[name]=pd.DataFrame(mean_embedded_test[name])
df_embedding_tr={}
df_embedding_te={}
for name in w2v:
    df_embedding_tr[name]=df_embed_tr[name].reset_index()
    df_embedding_te[name]=df_embed_te[name].reset_index()
    df_embedding_tr[name]=df_embedding_tr[name].rename(columns={"index":"ID"})
    df_embedding_te[name]=df_embedding_te[name].rename(columns={"index":"ID"})
train_w2v={}
test_w2v={}
for name in w2v:
    train_w2v[name]=pd.concat((train_cl,df_embedding_tr[name]),axis=1)
    test_w2v[name]=pd.concat((test_cl,df_embedding_te[name]),axis=1)


KeyboardInterrupt: 

In [ ]:
for name in w2v:
    train_w2v[name].to_csv("checkpoints_databases/nw_working_train_"+name+".csv",index=False)
    test_w2v[name].to_csv("checkpoints_databases/nw_working_test_"+name+".csv",index=False)

# Now TFIDF+300tsvd

In [16]:
tfidf = TfidfVectorizer(
        min_df=10, max_features=10000, strip_accents=None, lowercase = False,
        analyzer='word', token_pattern=r'\w+', ngram_range=(1,3), use_idf=True,
        smooth_idf=True, sublinear_tf=True
        ).fit(train["Text"])

X_train_text = tfidf.transform(train["Text"])
X_test_text = tfidf.transform(test["Text"])

In [38]:
tsvd_train= {}
tsvd_test={}
list_comp=[100,200,300]
dic_svd={}
for comp in list_comp:
    dic_svd[str(comp)]=TruncatedSVD(n_components=comp,n_iter=25,random_state=26)
for svd in dic_svd:
    tsvd_train[svd]=dic_svd[svd].fit_transform(X_train_text)
    tsvd_test[svd]=dic_svd[svd].transform(X_test_text)
X_train=pd.DataFrame()
X_test=pd.DataFrame()
for n in dic_svd:
    for i in range(int(n)):
        X_train['tsvd_' +str(n)+"_"+str(i)] = tsvd_train[n][:, i]
        X_test['tsvd_' +str(n)+"_"+str(i)] = tsvd_test[n][:, i]

In [40]:
X_train_100=X_train.iloc[:,:100]
X_train_200=X_train.iloc[:,100:300]
X_train_300=X_train.iloc[:,300:600]
X_test_100=X_test.iloc[:,:100]
X_test_200=X_test.iloc[:,100:300]
X_test_300=X_test.iloc[:,300:600]
dic_train={}
dic_test={}
dic_train["tsvd_100"]=pd.concat((train_cl, X_train_100),axis=1)
dic_test["tsvd_100"]=pd.concat((test_cl,X_test_100),axis=1)
dic_train["tsvd_200"]=pd.concat((train_cl, X_train_200),axis=1)
dic_test["tsvd_200"]=pd.concat((test_cl,X_test_200),axis=1)
dic_train["tsvd_300"]=pd.concat((train_cl, X_train_300),axis=1)
dic_test["tsvd_300"]=pd.concat((test_cl,X_test_300),axis=1)


In [67]:
for name in dic_train:
    dic_train[name].to_csv("checkpoints_databases/nw_working_train_tfidf_"+name+".csv",index=False)
    dic_test[name].to_csv("checkpoints_databases/nw_working_test_tfidf_"+name+".csv",index=False)

# Now Doc2Vec

In [19]:
def constructLabeledSentences(data):
    sentences=[]
    for index, row in data.iteritems():
        sentences.append(LabeledSentence(utils.to_unicode(row).split(), ['Text' + '_%s' % str(index)]))
    return sentences

train_sentences = constructLabeledSentences(train['Text'])
test_sentences = constructLabeledSentences(test['Text'])

Text_dim=[100,200,300]
d2v_train={}
d2v_test={}
for size in Text_dim:
    d2v_train["d2v_"+str(size)] = Doc2Vec(min_count=1, window=10, size=size, sample=1e-4, negative=5, workers=-1, iter=5,seed=26)
    d2v_train["d2v_"+str(size)].build_vocab(train_sentences)
    d2v_train["d2v_"+str(size)].train(train_sentences, total_examples=d2v_train["d2v_"+str(size)].corpus_count,
                                      epochs=d2v_train["d2v_"+str(size)].iter)
for size in Text_dim:
    d2v_test["d2v_"+str(size)] = Doc2Vec(min_count=1, window=10, size=size, sample=1e-4, negative=5, workers=-1, iter=5,seed=26)
    d2v_test["d2v_"+str(size)].build_vocab(test_sentences)
    d2v_test["d2v_"+str(size)].train(test_sentences, total_examples=d2v_test["d2v_"+str(size)].corpus_count,
                                 epochs=d2v_test["d2v_"+str(size)].iter)

In [20]:
d2v_train_arrays={}
d2v_test_arrays={}
d2v_train_arrays["d2v_100"] = np.zeros((len(train), 100))
d2v_test_arrays["d2v_100"] = np.zeros((len(test), 100))
d2v_train_arrays["d2v_200"] = np.zeros((len(train), 200))
d2v_test_arrays["d2v_200"] = np.zeros((len(test), 200))
d2v_train_arrays["d2v_300"] = np.zeros((len(train), 300))
d2v_test_arrays["d2v_300"] = np.zeros((len(test), 300))


for n_train,n_test in zip(range(len(train)),range(len(test))):
    d2v_train_arrays["d2v_100"][n_train] = d2v_train["d2v_100"].docvecs['Text_'+str(n_train)]
    d2v_test_arrays["d2v_100"][n_test] = d2v_test["d2v_100"].docvecs['Text_'+str(n_test)]
    d2v_train_arrays["d2v_200"][n_train] = d2v_train["d2v_200"].docvecs['Text_'+str(n_train)]
    d2v_test_arrays["d2v_200"][n_test] = d2v_test["d2v_200"].docvecs['Text_'+str(n_test)]
    d2v_train_arrays["d2v_300"][n_train] = d2v_train["d2v_300"].docvecs['Text_'+str(n_train)]
    d2v_test_arrays["d2v_300"][n_test] = d2v_test["d2v_300"].docvecs['Text_'+str(n_test)]
    


In [68]:
X_train_d2v={}
X_test_d2v={}
for name in d2v_train:
    X_train_d2v[name]=pd.concat((train_cl, pd.DataFrame(d2v_train_arrays[name])),axis=1)
    X_test_d2v[name]=pd.concat((test_cl, pd.DataFrame(d2v_test_arrays[name])),axis=1)
    print(X_train_d2v[name].shape,X_test_d2v[name].shape)
for name in X_train_d2v:
    X_train_d2v[name].to_csv("checkpoints_databases/nw_working_train_"+name+".csv",index=False)
    X_test_d2v[name].to_csv("checkpoints_databases/nw_working_test_"+name+".csv",index=False)
    

(3689, 138) (986, 138)
(3689, 238) (986, 238)
(3689, 338) (986, 338)
